<a href="https://colab.research.google.com/github/cimbelli/Surface-Heat-Islands/blob/main/temperatura_stats_mese_capoluoghi-regione_1.19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stima delle temperatura massima mensile nei capoluoghi di regione

In [4]:
import os
import ee
import pandas as pd
import json
import geopandas as gpd
import requests

ee.Authenticate()
ee.Initialize()

import wget
from rasterstats import zonal_stats
import calendar

# scegli quale statistica estrarre per sezione sulla base delle temperature massime estratte per pixel
statistiche = "min mean max median"
stat_map = "median"

class_intervals = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 100]
anni = [2019, 2020, 2021, 2022, 2023]
mesi = [1,2,3,4,5,6,7,8,9,10,11,12]

In [6]:
import folium
from folium import plugins

def last_day_of_month(year, month):
    _, last_day = calendar.monthrange(year, month)
    return last_day
    
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
    ).add_to(self)

def scarica(lyr, name, aoi):
    url = lyr.getDownloadUrl({
        'name': name[:-4],
        'bands': ['ST'],
        'region': aoi,
        'scale': 30,
        'format': 'GEO_TIFF'
    })
    response = requests.get(url)
    with open(name, 'wb') as fd:
        fd.write(response.content)
    return None

# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True,
        show=False
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True,
        show=False
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True,
        show=False
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True,
        show=False
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True,
        show=False
    )
}

In [7]:
# carica lo shape dei capoluoghi di regione
nomecap = 'caporeg.geojson'
if not(os.path.isfile(nomecap)):
  #wget.download('https://github.com/cimbelli/Surface-Heat-Islands/raw/5006ef2e37091eedf421f65ed9cbf80c15ccbdfd/' + nomecap)
  wget.download('https://raw.githubusercontent.com/cimbelli/Surface-Heat-Islands/main/' + nomecap)
g = open(nomecap)
cc = json.load(g)

gdf = gpd.read_file(nomecap)
centro = gdf.to_crs(epsg=3857).centroid.to_crs(gdf.crs)
#gdf["lon"] = gdf.to_crs(epsg=3857).centroid.to_crs(gdf.crs) #gdf["geometry"].to_crs(4326).centroid.x
#gdf["lat"] = gdf["geometry"].centroid.y
gdf["lon"] = centro.x
gdf["lat"] = centro.y

In [ ]:
for i in range(len(cc['features'])):           # for all the records

    reg = cc['features'][i]['properties']['COD_REG']
    reg1 = ('0' if reg < 10 else '') + str(reg)
    id1 = cc['features'][i]['properties']['PRO_COM']
    procom1 = '0' * (6-len(str(id1))) + str(id1)

    coords = cc['features'][i]['geometry']['coordinates']
    fc = ee.FeatureCollection(cc['features'])
    comune = cc['features'][i]['properties']['COMUNE']
    print('========',comune ,'========')

    nomefilesez = 'sez11_' + procom1 +'.geojson'
    if not(os.path.isfile(nomefilesez)):
      urlfilesez = 'https://raw.githubusercontent.com/cimbelli/Surface-Heat-Islands/main/sez11/' + nomefilesez
      wget.download(urlfilesez)
    g1 = open(nomefilesez)
    df_sez11 = gpd.read_file(g1)
    df_sez11['PRO_COM'] = df_sez11['PRO_COM'].astype(int)
    df_sez11['SEZ'] = df_sez11['SEZ'].astype(int)

    df_sez11[["P1","P14","P15","P28","P29"]] = df_sez11[["P1","P14","P15","P28","P29"]].fillna(0)
    df_sez11.rename(columns={'P1': 'Tot_residenti'}, inplace=True)#.astype('int')
    df_sez11['Tot_residenti'] = df_sez11['Tot_residenti'].astype('int')
    df_sez11['Bambini_0-9'] = (df_sez11['P14'].astype('int') + df_sez11['P15'].astype('int')).astype('int')
    df_sez11['Anziani_>70'] = (df_sez11['P28'].astype('int') + df_sez11['P29'].astype('int')).astype('int')

    lat = gdf[gdf['PRO_COM']== id1]['lat'].values[0]
    lon = gdf[gdf['PRO_COM']== id1]['lon'].values[0]

    minx = gdf[gdf['PRO_COM']== id1].geometry.bounds.minx.values[0]
    maxx = gdf[gdf['PRO_COM']== id1].geometry.bounds.maxx.values[0]
    miny = gdf[gdf['PRO_COM']== id1].geometry.bounds.miny.values[0]
    maxy = gdf[gdf['PRO_COM']== id1].geometry.bounds.maxy.values[0]

    region = ee.Geometry.BBox(minx, miny, maxx, maxy)
    centro1 = df_sez11.to_crs(epsg=3857).centroid.to_crs(df_sez11.crs)

    for anno in anni:
        anno1 = str(anno)[-2:]
        print('---',anno,'---')

        try:
          del sdf, df_o
        except:
          pass

        for mese in mesi:
            #mese2 = ('0' + str(mese+1)) if mese < 9 else str(mese+1)
            mese1 = ('0' + str(mese)) if mese < 9 else str(mese)
            nome_img = 'ST_' + comune + '_'+ str(anno) + '_'+ str(mese1) + '.tif'
            print(mese, end='..') 
            if not(os.path.isfile(nome_img)):
    
                L9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(str(anno) + '-' + mese1 +'-01', str(anno) + '-' + mese1 + '-' +
                                                                             str(last_day_of_month(anno, mese)))
                L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(str(anno) + '-' + mese1 +'-01', str(anno) + '-' + mese1 + '-' +
                                                                             str(last_day_of_month(anno, mese)))
                bands = ['ST_B10', 'QA_PIXEL']
                bandName = ['ST', 'QA_PIXEL']
                filtered_L9 = L9.filter(ee.Filter.lt('CLOUD_COVER', 40)).select(bands, bandName)#.map(cloudMask)
                filtered_L8 = L8.filter(ee.Filter.lt('CLOUD_COVER', 40)).select(bands, bandName)#.map(cloudMask)
                LandsatColl = filtered_L8.merge(filtered_L9)
                LandsatColl = LandsatColl.select('ST').filterBounds(region)
    
                if LandsatColl.size().getInfo() > 0:
                  lst = LandsatColl.max().clip(fc)
                  thermal = lst.select('ST.*').multiply(0.00341802).add(149.0).subtract(273.15)
                  n_img = LandsatColl.size().getInfo()
    
                  # zonal stats per sezioni
    #              print('scarico immagine', comune, anno, "n° acquisizioni", n_img)
    
                  scarica(thermal, nome_img, region)
                else:
                  print('nessuna immagine nel periodo selezionato')
                  continue
            else:
                print(nome_img, 'già presente')
    
    # calcola le statistiche zonali per anno
            stats = zonal_stats(df_sez11, nome_img, stats=statistiche)
            sdf = pd.DataFrame(stats)
            sdf = sdf.round(2)
            sdf.rename(columns={'min':'min'+anno1+mese1,'max':'max'+anno1+mese1,'median':'median'+anno1+mese1,'mean':'mean'+anno1+mese1}, inplace=True)
            sdf['n_img'+anno1+mese1] = n_img
            sdf['class'+anno1+mese1] = pd.cut(sdf[stat_map + anno1 + mese1], bins=class_intervals, labels=False)
            df_sez11 = pd.concat([df_sez11, sdf], axis=1)
            stats.clear()
    
            sdf2 = df_sez11.drop(['COD_REG','PRO_COM','P14','P15','P28','P29'], axis=1)
            sdf3 = sdf2.drop(['geometry'], axis=1)
            sdf3.to_excel(comune + '_stats.xlsx', index=False)




======== Milano ========
--- 2019 ---
1..2..3..

In [ ]:
sdf

In [ ]:
        # Crea la mappa per il comune e in estate
        m = folium.Map(location=(centro1.y[0],centro1.x[0] ), tiles="cartodb positron", zoom_start=12)
        folium.Choropleth(
            geo_data=sdf2,
            name ="Temperature SUPERFICIALI per sezione 2011 - " + comune + '-' + 'estate'+ str(anno),
            data=sdf2,
            columns=["SEZ", stat_map + anno1,"Tot_residenti","Bambini_0-9","Anziani_>70"],
            key_on="feature.properties.SEZ",
            fill_color= 'RdYlGn_r', #'YlOrRd',
            bins=[20, 25, 30, 35, 40, 45, 50, 55, 60, 100],
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name="Temperatura (" + stat_map + ") superficiale estiva per sezioni 2011",
        ).add_to(m)

        style_function = lambda x: {'fillColor': '#ffffff', 'color':'#000000', 'fillOpacity': 0.1, 'weight': 0.1}
        SuburbName = folium.features.GeoJson(sdf2, style_function=style_function,
            control=False, tooltip=folium.features.GeoJsonTooltip(
                fields=['SEZ', stat_map + anno1, "Tot_residenti","Bambini_0-9","Anziani_>70"],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
            )
        )
        m.add_child(SuburbName)
        m.keep_in_front(SuburbName)
        basemaps['Google Satellite'].add_to(m)

        folium.LayerControl().add_to(m)
        m.save(comune + '_' + str(anno) + '.html')


In [ ]:
# scaricare i file html e xlsx sul pc locale
from google.colab import files
import glob

html_list = glob.glob('*.html')
xlsx_list = glob.glob('*.xlsx')

for file_h in html_list:
  files.download(file_h)

for file_x in xlsx_list:
  files.download(file_x)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
for file_x in xlsx_list:
  files.download(file_x)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>